<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Datos
Se consumira los datos del artículo de wikipedia sobre "climate change" en ingles.

In [42]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Climate_change')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [43]:
# Demos un vistazo
article_text

'\n\n\ncontemporary climate change includes both global warming and its impacts on earth\'s weather patterns. there have been previous periods of climate change, but the current changes are distinctly more rapid and not due to natural causes.[2] instead, they are caused by the emission of greenhouse gases, mostly carbon dioxide (co2) and methane. burning fossil fuels for energy use creates most of these emissions. certain agricultural practices, industrial processes, and forest loss are additional sources.[3] greenhouse gases are transparent to sunlight, allowing it through to heat the earth\'s surface. when the earth emits that heat as infrared radiation the gases absorb it, trapping the heat near the earth\'s surface. as the planet heats up it causes changes like the loss of sunlight-reflecting snow cover, amplifying global warming.[4]\ndue to climate change, deserts are expanding, while heat waves and wildfires are becoming more common.[5] increased warming in the arctic has contrib

In [44]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 57300


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [45]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [46]:
# Demos un vistazo
text

' contemporary climate change includes both global warming and its impacts on earth\'s weather patterns. there have been previous periods of climate change, but the current changes are distinctly more rapid and not due to natural causes. instead, they are caused by the emission of greenhouse gases, mostly carbon dioxide (co2) and methane. burning fossil fuels for energy use creates most of these emissions. certain agricultural practices, industrial processes, and forest loss are additional sources. greenhouse gases are transparent to sunlight, allowing it through to heat the earth\'s surface. when the earth emits that heat as infrared radiation the gases absorb it, trapping the heat near the earth\'s surface. as the planet heats up it causes changes like the loss of sunlight-reflecting snow cover, amplifying global warming. due to climate change, deserts are expanding, while heat waves and wildfires are becoming more common. increased warming in the arctic has contributed to melting pe

In [47]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 55587


### 3 - Dividir el texto en sentencias y en palabras

In [48]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [49]:
# Demos un vistazo
corpus[:10]

[" contemporary climate change includes both global warming and its impacts on earth's weather patterns.",
 'there have been previous periods of climate change, but the current changes are distinctly more rapid and not due to natural causes.',
 'instead, they are caused by the emission of greenhouse gases, mostly carbon dioxide (co2) and methane.',
 'burning fossil fuels for energy use creates most of these emissions.',
 'certain agricultural practices, industrial processes, and forest loss are additional sources.',
 "greenhouse gases are transparent to sunlight, allowing it through to heat the earth's surface.",
 "when the earth emits that heat as infrared radiation the gases absorb it, trapping the heat near the earth's surface.",
 'as the planet heats up it causes changes like the loss of sunlight-reflecting snow cover, amplifying global warming.',
 'due to climate change, deserts are expanding, while heat waves and wildfires are becoming more common.',
 'increased warming in the ar

In [50]:
# Demos un vistazo
words[:20]

['contemporary',
 'climate',
 'change',
 'includes',
 'both',
 'global',
 'warming',
 'and',
 'its',
 'impacts',
 'on',
 'earth',
 "'s",
 'weather',
 'patterns',
 '.',
 'there',
 'have',
 'been',
 'previous']

In [51]:
print("Vocabulario:", len(words))

Vocabulario: 9655


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [52]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. 

In [54]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 5.1 MB 4.4 MB/s 
     |████████████████████████████████| 140 kB 45.2 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 212 kB 57.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.7 MB/s 
     |████████████████████████████████| 272 kB 45.9 MB/s 
     |████████████████████████████████| 144 kB 63.4 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 68 kB 5.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████

In [55]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://42678.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


heat


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


flood


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f7a915c2d50>,
 'http://127.0.0.1:7860/',
 'https://42678.gradio.app')